# Ejercicio LOGS NASA
## Marina Ortín Fernández.
   ### ETL

In [8]:

from pyspark import SparkContext
sc = SparkContext()
from pyspark import SQLContext
sqlContext = SQLContext(sc)


from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt
from pyspark.sql.functions import udf, array
from pyspark.sql.types import StringType
from pyspark.sql.functions import lit, when, col
from pyspark.sql.functions import col
from pyspark.sql.functions import substring
from pyspark.sql.types import IntegerType

In [47]:
import urllib.request
f = urllib.request.urlretrieve("https://www.dropbox.com/s/73wr8xb5s6fdj7g/apache.access.log.PROJECT?dl=1", "apache.access.log.PROJECT")

In [53]:
datos = "./apache.access.log.PROJECT"
data = sc.textFile(datos)

In [54]:
data.count()

1043177

In [55]:
from datetime import datetime
def day_month(line):
    date_time = line[3]
    return datetime.strptime(date_time[:11], "%d/%b/%Y")

In [56]:
data.take(2)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0']

In [57]:
datosLimpio = data.map(lambda x: x.replace(" -", ' 0')).map(lambda x: x.split(" "))

In [61]:
row_data = datosLimpio.map(lambda p: Row(
    host = p[0], 
    vacia1 = int(p[1]),
    vacia2 = p[2],
    date = p[3],
    protocol = p[4],
    endpoint = p[6],
    version = p[7],
    response_code = int(p[-2]),
    size = p[-1]
    )
)


interactions_df = sqlContext.createDataFrame(row_data)



In [62]:
df = interactions_df.withColumn('date', regexp_replace('date', '[', ''))
df2 = interactions_df.withColumn("fecha", substring('date', 2, 11))

from pyspark.sql.types import IntegerType

df3 = df2.withColumn("size", df2["size"].cast(IntegerType()))

In [63]:
df4a = df3.drop('vacia1')
df4b = df4a.drop('vacia2')
df4 = df4b.drop('date')

In [64]:
df4.show()

+--------------------+--------------------+--------+-------------+-----+---------+-----------+
|            endpoint|                host|protocol|response_code| size|  version|      fecha|
+--------------------+--------------------+--------+-------------+-----+---------+-----------+
|/shuttle/missions...|   in24.inetnebr.com|  00400]|          200| 1839|HTTP/1.0"|01/Aug/1995|
|                   /|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995|
|/images/ksclogo-m...|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995|
|/images/MOSAIC-lo...|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995|
|/images/USA-logos...|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995|
|/images/launch-lo...|ix-esc-ca2-07.ix....|  00400]|          200| 1713|HTTP/1.0"|01/Aug/1995|
|/images/WORLD-log...|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995|
|/history/skylab/s...|slppp6.intermind.net|  00400

In [65]:
split_col = split(df4['fecha'], '/')
df5 = df4.withColumn('dia', split_col.getItem(0))
df6 = df5.withColumn('mes', split_col.getItem(1))
df7 = df6.withColumn('año', split_col.getItem(2))
df7 = df7.na.fill(0)

In [66]:
df7.show()

+--------------------+--------------------+--------+-------------+-----+---------+-----------+---+---+----+
|            endpoint|                host|protocol|response_code| size|  version|      fecha|dia|mes| año|
+--------------------+--------------------+--------+-------------+-----+---------+-----------+---+---+----+
|/shuttle/missions...|   in24.inetnebr.com|  00400]|          200| 1839|HTTP/1.0"|01/Aug/1995| 01|Aug|1995|
|                   /|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995| 01|Aug|1995|
|/images/ksclogo-m...|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995| 01|Aug|1995|
|/images/MOSAIC-lo...|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995| 01|Aug|1995|
|/images/USA-logos...|     uplherc.upl.com|  00400]|          304|    0|HTTP/1.0"|01/Aug/1995| 01|Aug|1995|
|/images/launch-lo...|ix-esc-ca2-07.ix....|  00400]|          200| 1713|HTTP/1.0"|01/Aug/1995| 01|Aug|1995|
|/images/WORLD-log...|     u

### Pregunta 1. Mínimo,máximo y media del tamaño de peticiones (size)

In [67]:
size = df4.select('size')

In [70]:
size.agg({"size": "max"}).show()

+---------+
|max(size)|
+---------+
|  3421948|
+---------+



In [71]:
size.agg({"size": "min"}).show()

+---------+
|min(size)|
+---------+
|        0|
+---------+



In [72]:
size.agg({"size": "avg"}).show()

+------------------+
|         avg(size)|
+------------------+
|17531.555702435926|
+------------------+



### Pregunta 2. Número de peticiones de cada código de respuesta

In [73]:
peticiones = df4.groupby('response_code').agg(count('host').alias('cuenta')).show()

+-------------+------+
|response_code|cuenta|
+-------------+------+
|          304| 79824|
|          404|  6185|
|          500|     2|
|          501|    17|
|          403|    58|
|          200|940847|
|          302| 16244|
+-------------+------+



### Pregunta 3.Mostrar 20 hosts que han sido visitados más de 10 veces

In [74]:
from pyspark.sql.functions import col

hosts = df4.groupby('host').count().sort(col("count").desc()).show(20)

+--------------------+-----+
|                host|count|
+--------------------+-----+
|  edams.ksc.nasa.gov| 4034|
|piweba5y.prodigy.com| 3237|
|piweba4y.prodigy.com| 3043|
|piweba3y.prodigy.com| 2830|
|www-d1.proxy.aol.com| 2715|
|www-b3.proxy.aol.com| 2518|
|         news.ti.com| 2507|
|www-b2.proxy.aol.com| 2481|
|        163.206.89.4| 2478|
|www-c2.proxy.aol.com| 2438|
|www-c3.proxy.aol.com| 2400|
|www-d2.proxy.aol.com| 2371|
|www-d4.proxy.aol.com| 2356|
|www-b5.proxy.aol.com| 2354|
|www-b4.proxy.aol.com| 2297|
|www-d3.proxy.aol.com| 2284|
|www-a2.proxy.aol.com| 2238|
|www-c4.proxy.aol.com| 2207|
|www-c5.proxy.aol.com| 2198|
|www-c6.proxy.aol.com| 2181|
+--------------------+-----+
only showing top 20 rows



### Pregunta 4. Mostrar los 10 endpoints más visitados.

In [75]:
endpoints = df4.groupby('endpoint').count().sort(col("count").desc()).show(10)

+--------------------+-----+
|            endpoint|count|
+--------------------+-----+
|/images/NASA-logo...|59666|
|/images/KSC-logos...|50420|
|/images/MOSAIC-lo...|43831|
|/images/USA-logos...|43604|
|/images/WORLD-log...|43217|
|/images/ksclogo-m...|41267|
|           /ksc.html|28536|
|/history/apollo/i...|26766|
|/images/launch-lo...|24742|
|                   /|20175|
+--------------------+-----+
only showing top 10 rows



### Pregunta 5. Mostrar los 10 endpoints más visitados que no tienen código de respuesta 200.

In [76]:
endpointsNo200 = df4.filter(df4.response_code != '200').groupby('endpoint').count().sort(col("count").desc()).show(10)

+--------------------+-----+
|            endpoint|count|
+--------------------+-----+
|/images/NASA-logo...| 8761|
|/images/KSC-logos...| 7236|
|/images/MOSAIC-lo...| 5197|
|/images/USA-logos...| 5157|
|/images/WORLD-log...| 5020|
|/images/ksclogo-m...| 4728|
|/history/apollo/i...| 2907|
|/images/launch-lo...| 2811|
|                   /| 2199|
|/images/ksclogosm...| 1622|
+--------------------+-----+
only showing top 10 rows



### Pregunta 6. Calcular el número de hosts distintos

In [77]:
hostsDistintos = df4.select("host").distinct().count()
hostsDistintos

54507

### Pregunta 7. Calcular número de hosts distintos cada día.

In [85]:
#Muestra los host distintos de cada día del mes de Agosto
hostpordia = df7.groupby("dia").agg(countDistinct('host')).sort('dia').show()


+---+--------------------+
|dia|count(DISTINCT host)|
+---+--------------------+
| 01|                2582|
| 03|                3222|
| 04|                4190|
| 05|                2502|
| 06|                2537|
| 07|                4106|
| 08|                4406|
| 09|                4317|
| 10|                4523|
| 11|                4346|
| 12|                2864|
| 13|                2650|
| 14|                4454|
| 15|                4214|
| 16|                4340|
| 17|                4385|
| 18|                4168|
| 19|                2550|
| 20|                2560|
| 21|                4134|
+---+--------------------+
only showing top 20 rows



### Pregunta 9. Mostrar una lista de 40 endpoints distintos que generan un código de respuesta 404

In [199]:

endpointDistintos = df4.select("endpoint").distinct().show(40)

+--------------------+
|            endpoint|
+--------------------+
|/biomed/wwwicons/...|
|    /mdss/s_md-2.gif|
|/shuttle/missions...|
|/history/apollo/a...|
|/shuttle/missions...|
|/shuttle/technolo...|
|/cgi-bin/imagemap...|
|/shuttle/missions...|
|/shuttle/resource...|
|/shuttle/missions...|
|/history/apollo/a...|
|/pub/winvn/readme...|
|/shuttle/missions...|
|/shuttle/resource...|
|/shuttle/missions...|
|/cgi-bin/imagemap...|
|/shuttle/technolo...|
|/shuttle/missions...|
|/images/dual-pad.gif|
|/shuttle/missions...|
|/shuttle/missions...|
|/facilities/tour....|
|/shuttle/countdow...|
|          /ksc.html/|
|      /mdss/dcy2.gif|
|/shuttle/missions...|
|/history/apollo/a...|
|/history/apollo/s...|
|/history/apollo/a...|
|/shuttle/missions...|
|/history/apollo/a...|
|/history/apollo/a...|
|/history/apollo/i...|
|/history/apollo/a...|
|          /ksc.html"|
|/shuttle/technolo...|
|/elv/ATLAS_CENTAU...|
|/software/winvn/f...|
|/elv/DELTA/uncons...|
|/shuttle/missions...|
+----------

### Pregunta 10. Mostrar top 25 endpoints que más códigos de respuesta 404 generan.

In [80]:
endpoints404 = df4.filter(df4.response_code == '404').groupby('endpoint').count().sort(col("count").desc()).show(25)

+--------------------+-----+
|            endpoint|count|
+--------------------+-----+
|/pub/winvn/readme...|  633|
|/pub/winvn/releas...|  494|
|/shuttle/missions...|  430|
|/images/nasa-logo...|  319|
|/elv/DELTA/uncons...|  178|
|/shuttle/missions...|  154|
|/history/apollo/s...|  146|
|/images/crawlerwa...|  120|
|/://spacelink.msf...|  117|
|/history/apollo/p...|  100|
|/history/apollo/a...|   97|
|/images/Nasa-logo...|   85|
|/shuttle/resource...|   63|
|/history/apollo/i...|   62|
| /images/lf-logo.gif|   59|
|/shuttle/resource...|   56|
|/shuttle/resource...|   54|
|         /robots.txt|   53|
|     /elv/new01.gif>|   43|
|/history/apollo/p...|   38|
|                /pub|   36|
|               /pub/|   36|
|/history/apollo/s...|   35|
|/history/apollo/s...|   32|
|/shuttle/countdow...|   31|
+--------------------+-----+
only showing top 25 rows



### Pregunta 11. Top 5 de días que generaron código 404

In [81]:
top404 = df4.select('fecha','response_code').filter(df4.response_code == '404')
response = top404.groupby('fecha','response_code').count().show()


+-----------+-------------+-----+
|      fecha|response_code|count|
+-----------+-------------+-----+
|09/Aug/1995|          404|  279|
|06/Aug/1995|          404|  372|
|18/Aug/1995|          404|  255|
|13/Aug/1995|          404|  216|
|21/Aug/1995|          404|  305|
|12/Aug/1995|          404|  195|
|14/Aug/1995|          404|  287|
|16/Aug/1995|          404|  258|
|01/Aug/1995|          404|  243|
|04/Aug/1995|          404|  346|
|20/Aug/1995|          404|  312|
|15/Aug/1995|          404|  326|
|03/Aug/1995|          404|  303|
|22/Aug/1995|          404|  288|
|17/Aug/1995|          404|  269|
|05/Aug/1995|          404|  234|
|10/Aug/1995|          404|  314|
|08/Aug/1995|          404|  381|
|07/Aug/1995|          404|  532|
|11/Aug/1995|          404|  263|
+-----------+-------------+-----+
only showing top 20 rows

